# Template Quest Notebook

In [1]:
# Import neccessary modules, add to this cell as needed
# Provided PySpark examples
import pandas as pd
import requests
import whois
import json
from datetime import datetime, UTC
import os
from dotenv import load_dotenv
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, LongType

## Part 1: Load the Sample Dataset

In [2]:
# Initiate a new Spark session and set the case sensitivity option
spark = (
    SparkSession.builder
        .appName("cyberquest")
        .getOrCreate()
)
spark.conf.set("spark.sql.caseSensitive", True)

In [3]:
# TODO: Load the raw data
df_bronze = spark.read.json("data/sysmon_spearphish_cribl.json")
# A short preview of the data is always helpful for a higher-level understanding of the data we will be dealing with.
print(f"Total records loaded: {df_bronze.count()}")
df_bronze.printSchema()
df_bronze.limit(3).toPandas()  # Found toPandas() in apache spark doc, found this more visually readable than default .show()

Total records loaded: 28017
root
 |-- Computer: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- EventCode: string (nullable = true)
 |-- User: string (nullable = true)
 |-- _raw: string (nullable = true)
 |-- _time: double (nullable = true)
 |-- cribl_breaker: string (nullable = true)
 |-- cribl_pipe: string (nullable = true)
 |-- host: string (nullable = true)
 |-- source: string (nullable = true)



,Computer,Description,EventCode,User,_raw,_time,cribl_breaker,cribl_pipe,host,source
0,win-host-ctus-attack-range-212,None,23,NT AUTHORITY\SYSTEM,"{""Name"":""'Microsoft-Windows-Sysmon'"",""Guid"":""'...",1.674818e+09,fallback,Splunk_UF_Windows_XML_WEC_WEF_Sysmon,f2c75b47cbb7,/tmp/cribldata/in/sysmon_office_doc_abuses_rel...
1,win-dc-ctus-attack-range-460.attackrange.local,None,23,NT AUTHORITY\SYSTEM,"{""Name"":""'Microsoft-Windows-Sysmon'"",""Guid"":""'...",1.674818e+09,fallback,Splunk_UF_Windows_XML_WEC_WEF_Sysmon,f2c75b47cbb7,/tmp/cribldata/in/sysmon_office_doc_abuses_rel...
2,win-host-ctus-attack-range-212,None,23,NT AUTHORITY\SYSTEM,"{""Name"":""'Microsoft-Windows-Sysmon'"",""Guid"":""'...",1.674818e+09,fallback,Splunk_UF_Windows_XML_WEC_WEF_Sysmon,f2c75b47cbb7,/tmp/cribldata/in/sysmon_office_doc_abuses_rel...


### Dataset Overview

It is always a good practice to understand the data that we will be working with, which is something analogous to how we identify assets and try to get visibility on the data classification before we start any assessment.

### Source:
- Windows endpoints --> Sysmon Logs (DNS) --> Splunk UF --> Cribl --> SIEM / Data Lake

In simple words, the dataset contains Windows Sysmon logs collected from Windows endpoints that were shipped via Splunk Universal Forwarder to Cribl, which orchestrates, normalizes, and routes them to the detection platform SIEM / Data Lake.


**File:** `data/sysmon_spearphish_cribl.json`  
**Format:** JSONL (low memory requirements, best for processing and streaming line by line, fault tolerant)  
**Time Range:** January 27, 2023 | 11: 19: 47 to 11: 34: 46 | Approximately 15 mins  
**Environment:** Looks like a simulated attack range (attackrange.local)

### Data Structure

Each record has two layers:
- Outer (added by Cribl during forwarding)
- Inner (_raw: which contains the actual sysmon event data)

### Some Initial Considerations

- EventCode (22) is basically a DNS event, and this will be a key event considering the provided hypothesis.
- Reason: Captures important fields like:
    - 'Image' (gives info on which process initiated the DNS lookup),
    - 'QueryName' (gives info on the domain that was queried),
    - 'QueryResults' (gives info on what IPs it resolved to)  


This is a good starting point for us. Under normal circumstances, Microsoft Office applications like winword.exe or excel.exe shouldn’t be generating DNS requests on their own. So when we observe that behavior, it stands out as a potential anomaly. It may indicate that a malicious macro was executed, which then initiated outbound communication, possibly marking the early stages of C2 beaconing.


There are two ways of parsing the data from the JSON:
1) Using get_json_object
2) Using from_json

Decided to choose the **get_json_object** over from_json as there are multiple EventCode with completely different fields inside _raw. Using the get_json_object would allow me to extract the fields that we need rather than defining one massive schema (required for from_json) that would most probably return nulls for major fields, as _raw is a mix of EventCodes

In [4]:
# TODO: Parse the raw data into something relevant and usable

df_silver = df_bronze.select(
    # Extracting the outer
    F.col("Computer").alias("computer"),
    F.col("EventCode").alias("event_code"),
    F.col("User").alias("user"),
    F.col("_time").alias("epoch_time"),
    F.col("host").alias("cribl_host"),
    F.col("source").alias("log_source"),

    # Parsing key fields from the inner (_raw) 
    F.get_json_object(F.col("_raw"), "$.UtcTime").alias("event_time"),
    F.get_json_object(F.col("_raw"), "$.Image").alias("process_image"),
    F.get_json_object(F.col("_raw"), "$.ProcessId").alias("process_id"),
    F.get_json_object(F.col("_raw"), "$.ProcessGuid").alias("process_guid"),
    F.get_json_object(F.col("_raw"), "$.QueryName").alias("dns_query"),
    F.get_json_object(F.col("_raw"), "$.QueryResults").alias("dns_results"),
    F.get_json_object(F.col("_raw"), "$.Channel").alias("channel"),
    F.get_json_object(F.col("_raw"), "$.RuleName").alias("rule_name"),
)

# Creating a SQL view will be used in downstream detection
df_silver.createOrReplaceTempView("silver_records")

print(f"Silver layer records: {df_silver.count()}")
df_silver.printSchema()

Silver layer records: 28017
root
 |-- computer: string (nullable = true)
 |-- event_code: string (nullable = true)
 |-- user: string (nullable = true)
 |-- epoch_time: double (nullable = true)
 |-- cribl_host: string (nullable = true)
 |-- log_source: string (nullable = true)
 |-- event_time: string (nullable = true)
 |-- process_image: string (nullable = true)
 |-- process_id: string (nullable = true)
 |-- process_guid: string (nullable = true)
 |-- dns_query: string (nullable = true)
 |-- dns_results: string (nullable = true)
 |-- channel: string (nullable = true)
 |-- rule_name: string (nullable = true)



In [5]:
# Provided PySpark Example
df_silver.limit(5).toPandas()

,computer,event_code,user,epoch_time,cribl_host,log_source,event_time,process_image,process_id,process_guid,dns_query,dns_results,channel,rule_name
0,win-host-ctus-attack-range-212,23,NT AUTHORITY\SYSTEM,1.674818e+09,f2c75b47cbb7,/tmp/cribldata/in/sysmon_office_doc_abuses_rel...,2023-01-27 11:19:47.591,C:\Program Files\SplunkUniversalForwarder\bin\...,3764,{72106695-9B97-63D3-6D00-00000000BD02},None,None,Microsoft-Windows-Sysmon/Operational,-
1,win-dc-ctus-attack-range-460.attackrange.local,23,NT AUTHORITY\SYSTEM,1.674818e+09,f2c75b47cbb7,/tmp/cribldata/in/sysmon_office_doc_abuses_rel...,2023-01-27 11:19:47.410,C:\Program Files\SplunkUniversalForwarder\bin\...,3928,{45AAC21C-9BAA-63D3-7B00-00000000BC02},None,None,Microsoft-Windows-Sysmon/Operational,-
2,win-host-ctus-attack-range-212,23,NT AUTHORITY\SYSTEM,1.674818e+09,f2c75b47cbb7,/tmp/cribldata/in/sysmon_office_doc_abuses_rel...,2023-01-27 11:19:47.008,C:\Program Files\Amazon\SSM\amazon-ssm-agent.exe,1880,{72106695-9B85-63D3-1B00-00000000BD02},None,None,Microsoft-Windows-Sysmon/Operational,-
3,win-dc-ctus-attack-range-460.attackrange.local,10,NaN,1.674818e+09,f2c75b47cbb7,/tmp/cribldata/in/sysmon_office_doc_abuses_rel...,2023-01-27 11:19:47.238,NaN,NaN,NaN,None,None,Microsoft-Windows-Sysmon/Operational,-
4,win-dc-ctus-attack-range-460.attackrange.local,10,NaN,1.674818e+09,f2c75b47cbb7,/tmp/cribldata/in/sysmon_office_doc_abuses_rel...,2023-01-27 11:19:47.061,NaN,NaN,NaN,None,None,Microsoft-Windows-Sysmon/Operational,-


In [6]:
# Provided PySpark SQL Example
spark.sql("""
SELECT *
FROM silver_records
LIMIT 5
""").toPandas()

,computer,event_code,user,epoch_time,cribl_host,log_source,event_time,process_image,process_id,process_guid,dns_query,dns_results,channel,rule_name
0,win-host-ctus-attack-range-212,23,NT AUTHORITY\SYSTEM,1.674818e+09,f2c75b47cbb7,/tmp/cribldata/in/sysmon_office_doc_abuses_rel...,2023-01-27 11:19:47.591,C:\Program Files\SplunkUniversalForwarder\bin\...,3764,{72106695-9B97-63D3-6D00-00000000BD02},None,None,Microsoft-Windows-Sysmon/Operational,-
1,win-dc-ctus-attack-range-460.attackrange.local,23,NT AUTHORITY\SYSTEM,1.674818e+09,f2c75b47cbb7,/tmp/cribldata/in/sysmon_office_doc_abuses_rel...,2023-01-27 11:19:47.410,C:\Program Files\SplunkUniversalForwarder\bin\...,3928,{45AAC21C-9BAA-63D3-7B00-00000000BC02},None,None,Microsoft-Windows-Sysmon/Operational,-
2,win-host-ctus-attack-range-212,23,NT AUTHORITY\SYSTEM,1.674818e+09,f2c75b47cbb7,/tmp/cribldata/in/sysmon_office_doc_abuses_rel...,2023-01-27 11:19:47.008,C:\Program Files\Amazon\SSM\amazon-ssm-agent.exe,1880,{72106695-9B85-63D3-1B00-00000000BD02},None,None,Microsoft-Windows-Sysmon/Operational,-
3,win-dc-ctus-attack-range-460.attackrange.local,10,NaN,1.674818e+09,f2c75b47cbb7,/tmp/cribldata/in/sysmon_office_doc_abuses_rel...,2023-01-27 11:19:47.238,NaN,NaN,NaN,None,None,Microsoft-Windows-Sysmon/Operational,-
4,win-dc-ctus-attack-range-460.attackrange.local,10,NaN,1.674818e+09,f2c75b47cbb7,/tmp/cribldata/in/sysmon_office_doc_abuses_rel...,2023-01-27 11:19:47.061,NaN,NaN,NaN,None,None,Microsoft-Windows-Sysmon/Operational,-


## Part 2: Detection Engineering

### My Approach

#### Provided Hypothesis:
- Adversaries may send spearphishing emails with a malicious attachment in an attempt to gain access to victim systems.
- A threat actor can leverage Microsoft Office applications against users on Windows endpoints through phishing to make potentially malicious web calls.
- Windows Endpoints forwarding DNS activity via Sysmon logs to a data orchestration platform like Cribl would be utilized to identify this activity.

#### My Detection Logic:
- To prove the hypothesis, filtering for Sysmon EventCode 22 (DNS Query) events where the initiating process is a Microsoft Office application (Word, Excel, PowerPoint, etc) would be the starting point. Office apps like Word or Excel have no reason to query external domains on their own. If they are, it's a strong indicator that a malicious macro from a phishing email got executed and is beaconing out to a Command & Control server.

**Why only DNS?**

- In a real attack chain, after a user opens a phishing email and enables a macro, the very first thing the malware does is a DNS lookup, before any HTTP call, before any data exfiltration, before anything else.

#### Mapping to MITRE Attack (TTPs):
1) **Tactic: Initial Access (TA0001)** - Initial Access consists of techniques that use various entry vectors to gain their initial foothold within a network. Techniques used to gain a foothold include targeted spearphishing and exploiting weaknesses on public-facing web servers.

**Why this tactic:** There are two tactics under which we can classify phising one is Reconnaissance (TA0043), and another is Initial Access (TA0001). Basically, the aim of phishing in reconnaissance is most likely to elicit sensitive information that can be used during targeting, whereas the aim of phishing in initial access is to attempt to gain access to victim systems, which exactly matches our hypothesis.

- **Technique: Phishing: Spearphishing Attachment (T1566.001)** - Adversaries may send spearphishing emails with a malicious attachment in an attempt to gain access to victim systems. Spearphishing attachment is a specific variant of spearphishing. Spearphishing attachment is different from other forms of spearphishing in that it employs the use of malware attached to an email. All forms of spearphishing are electronically delivered social engineering targeted at a specific individual, company, or industry. In this scenario, adversaries attach a file to the spearphishing email and usually rely upon User Execution to gain execution. Spearphishing may also involve social engineering techniques, such as posing as a trusted source.

**Why this technique:** This exactly matches our hypothesis, where the threat actor can send spearphishing emails with a malicious attachment, which then is executed by the user via the relevant Microsoft Office application, which would potentially make malicious web calls.

- **Procedure Example: 2015 Ukraine Electric Power Attack (C0028)**: During the 2015 Ukraine Electric Power Attack, Sandworm Team obtained their initial foothold into many IT systems using Microsoft Office attachments delivered through phishing emails.

**Note:** Looking at this through the Cyber Kill Chain, the DNS query happens at the Command & Control phase. This is the moment where the malware is trying to establish a connection to the attacker's infrastructure. What's powerful about catching it here is that we're stopping the attack before anything bad actually happens. We can break the chain right at the point where the attacker needs to communicate.

2) **Tactic: Command and Control (TA0011)** - Command and Control consists of techniques that adversaries may use to communicate with systems under their control within a victim network. Adversaries commonly attempt to mimic normal, expected traffic to avoid detection. There are many ways an adversary can establish command and control with various levels of stealth depending on the victim’s network structure and defenses.

**Why this tactic:** Our hypothesis assumes that a threat actor could abuse Microsoft Office applications to trigger potentially malicious outbound web requests. If that is the case, the logical next step would be establishing a communication channel back to the attacker. Most often, this is done through Command and Control, which allows the attacker to issue follow-up instructions or payloads after the initial compromise.

- **Technique: Application Layer Protocol: DNS (T1071.004)** - Adversaries may communicate using the Domain Name System (DNS) application layer protocol to avoid detection/network filtering by blending in with existing traffic. Commands to the remote system, and often the results of those commands, will be embedded within the protocol traffic between the client and server.DNS beaconing may be used to send commands to remote systems via DNS queries.

**Why this technique:** This exactly matches our hypothesis, the relevant Microsoft Office application, which would potentially make malicious web calls to potentially establish a communication channel back to the attacker, this is done through Command and Control

- **Procedure Example: Anchor (S0504)**: Variants of Anchor can use DNS tunneling to communicate with C2.

### Justification for Selected Fields

- **event_time:** Included to establish the exact timeline of activity. Timing is critical when reconstructing an attack sequence and correlating follow-on behavior.

- **computer:** Identifies which host generated the event. This is necessary for scoping impact and determining whether activity is isolated or widespread.

- **user:** Provides execution context. Helps determine whether the activity occurred under an interactive user session or a system/service account.

- **process_image** Indicates which executable initiated the DNS request. This is the core behavioral signal, especially when filtering for Office applications.

- **dns_query** Captures the domain being resolved. This is the key indicator used for enrichment and C2 suspicion.

- **dns_results** Shows the resolved IP address, which helps validate outbound connections and supports correlation with network activity.

- **process_id** Allows correlation with other events (e.g., file creation, network connections) tied to the same process instance.

- **process_guid** Provides a unique identifier for the process across events, which is more reliable than PID alone for correlation.

- **event_code** Retained for clarity and validation, ensuring the dataset reflects the intended event type during downstream analysis.

In [7]:
df_detection = spark.sql("""
    SELECT
        event_time,
        computer,
        user,
        process_image,
        dns_query,
        dns_results,
        process_id,
        process_guid,
        event_code
    FROM silver_records
    WHERE event_code = '22'
    AND (
        LOWER(process_image) LIKE '%winword.exe%'
        OR LOWER(process_image) LIKE '%excel.exe%'
        OR LOWER(process_image) LIKE '%powerpnt.exe%'
        OR LOWER(process_image) LIKE '%outlook.exe%'
        OR LOWER(process_image) LIKE '%onenote.exe%'
        OR LOWER(process_image) LIKE '%mspub.exe%'
        OR LOWER(process_image) LIKE '%msaccess.exe%'
    )
    AND dns_query IS NOT NULL
    ORDER BY event_time DESC
""")

print(f"Detections: {df_detection.count()} DNS events found")
df_detection.toPandas()

Detections: 8 DNS events found


,event_time,computer,user,process_image,dns_query,dns_results,process_id,process_guid,event_code
0,2023-01-27 11:31:06.106,win-host-ctus-attack-range-212,None,C:\Program Files\Microsoft Office\root\Office1...,uci.cdn.office.net,type: 5 uci.cdn.office.net.edgekey.net;type: ...,4200,{72106695-B5C3-63D3-3D04-00000000BD02},22
1,2023-01-27 11:31:02.217,win-host-ctus-attack-range-212,None,C:\Program Files\Microsoft Office\root\Office1...,binaries.templates.cdn.office.net,type: 5 binaries.templates.cdn.office.net.edg...,4200,{72106695-B5C3-63D3-3D04-00000000BD02},22
2,2023-01-27 11:31:02.111,win-host-ctus-attack-range-212,None,C:\Program Files\Microsoft Office\root\Office1...,metadata.templates.cdn.office.net,type: 5 templatesmetadata.office.net;type: 5...,4200,{72106695-B5C3-63D3-3D04-00000000BD02},22
3,2023-01-27 11:31:00.390,win-host-ctus-attack-range-212,None,C:\Program Files\Microsoft Office\root\Office1...,messaging.lifecycle.office.com,type: 5 prod-custommessage.omexexternallfb.of...,4200,{72106695-B5C3-63D3-3D04-00000000BD02},22
4,2023-01-27 11:30:50.904,win-host-ctus-attack-range-212,None,C:\Program Files\Microsoft Office\root\Office1...,augloop.office.com,type: 5 augloop-prod.trafficmanager.net;type:...,4200,{72106695-B5C3-63D3-3D04-00000000BD02},22
5,2023-01-27 11:30:14.523,win-host-ctus-attack-range-212,None,C:\Program Files\Microsoft Office\root\Office1...,www.mediafire.com,::ffff:104.16.54.48;::ffff:104.16.53.48;,4200,{72106695-B5C3-63D3-3D04-00000000BD02},22
6,2023-01-27 11:30:13.961,win-host-ctus-attack-range-212,None,C:\Program Files\Microsoft Office\root\Office1...,support.content.office.net,type: 5 support.content.office.net.edgekey.ne...,4200,{72106695-B5C3-63D3-3D04-00000000BD02},22
7,2023-01-27 11:30:13.957,win-host-ctus-attack-range-212,None,C:\Program Files\Microsoft Office\root\Office1...,ecs.office.com,type: 5 ecs.office.trafficmanager.net;type: ...,4200,{72106695-B5C3-63D3-3D04-00000000BD02},22


### Findings

- 8 DNS events were found
- 7 seems to be legitimate, except 1, i.e., mediafire (a file sharing site), which potentially indicates a macro downloading a payload.

## Part 3: Additional Steps

### For the Data Normalization:

- There are 4 main or widely used data model frameworks:
  - ECS (Elastic Common Schema), which is widely used in industry and is considered best for Elasticsearch, log analytics or for SIEM.
  - OSSEM (Open Source Security Events Metadata), which focuses primarily on the documentation and standardization of security event logs from diverse data sources and operating systems.
  - OCSF (Open Cybersecurity Schema Framework), a collaborative effort by AWS, focuses on cloud, but is also vendor-neutral.
  - Splunk CIM (Common Information Model), if the data will be ingested into Splunk

**My choice: OSSEM (Open Source Security Events Metadata)**

**Why?**

- Because the data is Windows Sysmon logs, OSSEM maps security events to MITRE ATT&CK techniques, focusing heavily on Windows processes, network connections, and registry modifications. (Found Windows-specific resources on OSSEM much easier and simpler to use and understand)

### Part 3.1: Normalization

In [8]:
# Normalizing detection results to OSSEM (Open Source Security Events Metadata)

df_normalized_data = df_detection.select(

    # Event fields
    F.col("event_time").alias("event_creation_time"),     
    F.col("event_code").alias("event_id"),                 

    # Process fields
    F.col("process_guid").alias("process_guid"),            
    F.col("process_id").alias("process_id"),                
    F.col("process_image").alias("process_file_path"),      

    # DNS fields
    F.col("dns_query").alias("dns_query_name"),            
    F.col("dns_results").alias("dns_response_name"),        

    # Host and User fields
    F.col("computer").alias("hostname"),                    
    F.col("user").alias("user_name"),                       

)

df_normalized_data.createOrReplaceTempView("normalized_data")

print(f"Normalized data records: {df_normalized_data.count()}")
df_normalized_data.toPandas()

Normalized data records: 8


,event_creation_time,event_id,process_guid,process_id,process_file_path,dns_query_name,dns_response_name,hostname,user_name
0,2023-01-27 11:31:06.106,22,{72106695-B5C3-63D3-3D04-00000000BD02},4200,C:\Program Files\Microsoft Office\root\Office1...,uci.cdn.office.net,type: 5 uci.cdn.office.net.edgekey.net;type: ...,win-host-ctus-attack-range-212,None
1,2023-01-27 11:31:02.217,22,{72106695-B5C3-63D3-3D04-00000000BD02},4200,C:\Program Files\Microsoft Office\root\Office1...,binaries.templates.cdn.office.net,type: 5 binaries.templates.cdn.office.net.edg...,win-host-ctus-attack-range-212,None
2,2023-01-27 11:31:02.111,22,{72106695-B5C3-63D3-3D04-00000000BD02},4200,C:\Program Files\Microsoft Office\root\Office1...,metadata.templates.cdn.office.net,type: 5 templatesmetadata.office.net;type: 5...,win-host-ctus-attack-range-212,None
3,2023-01-27 11:31:00.390,22,{72106695-B5C3-63D3-3D04-00000000BD02},4200,C:\Program Files\Microsoft Office\root\Office1...,messaging.lifecycle.office.com,type: 5 prod-custommessage.omexexternallfb.of...,win-host-ctus-attack-range-212,None
4,2023-01-27 11:30:50.904,22,{72106695-B5C3-63D3-3D04-00000000BD02},4200,C:\Program Files\Microsoft Office\root\Office1...,augloop.office.com,type: 5 augloop-prod.trafficmanager.net;type:...,win-host-ctus-attack-range-212,None
5,2023-01-27 11:30:14.523,22,{72106695-B5C3-63D3-3D04-00000000BD02},4200,C:\Program Files\Microsoft Office\root\Office1...,www.mediafire.com,::ffff:104.16.54.48;::ffff:104.16.53.48;,win-host-ctus-attack-range-212,None
6,2023-01-27 11:30:13.961,22,{72106695-B5C3-63D3-3D04-00000000BD02},4200,C:\Program Files\Microsoft Office\root\Office1...,support.content.office.net,type: 5 support.content.office.net.edgekey.ne...,win-host-ctus-attack-range-212,None
7,2023-01-27 11:30:13.957,22,{72106695-B5C3-63D3-3D04-00000000BD02},4200,C:\Program Files\Microsoft Office\root\Office1...,ecs.office.com,type: 5 ecs.office.trafficmanager.net;type: ...,win-host-ctus-attack-range-212,None


### Part 3.2: Alert Table

#### My approach and thought process:

- Packaging detection results into a high-fidelity alert table
- Only escalating truly suspicious domains, known Microsoft Office telemetry domains are excluded to reduce false positives and alert fatigue
- There are two options to create Alert IDs: uuids or a SHA256 hash of key fields
  - Decided to go with SHA256 hash to ensure idempotency, i.e., prevents duplicates by ensuring the same event always produces the same alert ID when detection jobs run. In simple words, using SHA256, the alert_id remains the same after multiple runs.

In [9]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

df_alert = df_normalized_data.filter(
    # Excluding known legitimate Microsoft Office domains
    ~(
        F.col("dns_query_name").like("%.office.com%") |
        F.col("dns_query_name").like("%.office.net%") |
        F.col("dns_query_name").like("%.microsoft.com%") |
        F.col("dns_query_name").like("%.windows.com%") |
        F.col("dns_query_name").like("%.windowsupdate.com%")
    )
).select(

    # Alert Identity
    F.sha2(
        # Selected these 3 fields to uniquely create an alert id
        F.concat_ws("|",
            F.col("event_creation_time"),    # WHEN it hapenned
            F.col("hostname"),               # WHERE it hapenned
            F.col("dns_query_name")          # WHAT was queried
        ), 256
    ).alias("alert_id"),
    F.lit("Microsoft Office Application Suspicious DNS Query").alias("alert_name"),
    F.lit(
    "Microsoft Office process executed a DNS query to a non-Microsoft domain. This is indicative of a possible macro-enabled phishing document beaconing to a Command and Control server. Known Microsoft telemetry domains have been excluded to reduce false positives."
    ).alias("alert_description"),
    F.lit("HIGH").alias("alert_severity"),
    F.lit("NEW").alias("alert_status"),
    F.current_timestamp().alias("alert_creation_time"),

    # Investigation Context
    F.col("hostname"),
    F.col("user_name"),
    F.col("process_file_path"),
    F.col("dns_query_name"),
    F.col("dns_response_name"),

    # Source Traceability
    F.col("event_id"),
    F.col("event_creation_time"),
    F.lit("Microsoft-Windows-Sysmon").alias("log_provider"),
    F.lit("normalized_data").alias("detection_source"),
    
    # Threat Context, mapping to MITRE
    # OSSEM uses standard names as "technique_id","technique","tactic" if we had only one mapping, 
    # Researched the standard but could not find official field names for multiple technique mappings.
    # Decided to add a numeric suffix to each field for clarity and easier interpretation.
    F.lit("T1566.001").alias("technique_id_1"),
    F.lit("Spearphishing Attachment").alias("technique_1"),
    F.lit("Initial Access").alias("tactic_1"),
    F.lit("T1071.004").alias("technique_id_2"),
    F.lit("Application Layer Protocol: DNS").alias("technique_2"),
    F.lit("Command and Control").alias("tactic_2"),
    F.lit("Command & Control").alias("kill_chain_phase"),

    
)

df_alert.createOrReplaceTempView("alerts")

print(f"Alerts generated: {df_alert.count()}")
df_alert.toPandas()

Alerts generated: 1


,alert_id,alert_name,alert_description,alert_severity,alert_status,alert_creation_time,hostname,user_name,process_file_path,dns_query_name,dns_response_name,event_id,event_creation_time,log_provider,detection_source,technique_id_1,technique_1,tactic_1,technique_id_2,technique_2,tactic_2,kill_chain_phase
0,725c19b21612dfdf94eef4bdb1ab157fac495183925056984a3c4aaafbbfc53c,Microsoft Office Application Suspicious DNS Query,Microsoft Office process executed a DNS query to a non-Microsoft domain. This is indicative of a possible macro-enabled phishing document beaconing to a Command and Control server. Known Microsoft telemetry domains have been excluded to reduce false positives.,HIGH,NEW,2026-02-24 21:42:36.821105,win-host-ctus-attack-range-212,None,C:\Program Files\Microsoft Office\root\Office16\WINWORD.EXE,www.mediafire.com,::ffff:104.16.54.48;::ffff:104.16.53.48;,22,2023-01-27 11:30:14.523,Microsoft-Windows-Sysmon,normalized_data,T1566.001,Spearphishing Attachment,Initial Access,T1071.004,Application Layer Protocol: DNS,Command and Control,Command & Control


### Part 3.3: Enrichment

### My approach:

Decided to go with 3 threat intelligence sources that would provide the analyst triaging this alert with enough visibility:

1) **Whois:** WHOIS provides domain registration details such as creation date, registrar, and country. This helps the analyst determine whether the domain looks like newly created attacker infrastructure or something long-established and less suspicious.
2) **URLhaus:** URLhaus provides visibility into domains that have been observed distributing malware in real campaigns. This helps the analyst understand whether the domain has a history of abuse, even if it belongs to a legitimate hosting platform.
3) **VirusTotal:** VirusTotal provides aggregated detection results from multiple security vendors. This allows the analyst to gauge broader industry consensus and adjust their confidence level in the alert accordingly.

There is a long list of Threat Intel sources that could have been used, e.g. AlientVaultOTX, ThreatFox, PassiveDNS, etc.

**Reasoning behind threat intelligence sources selection:**

- Together they give a well-rounded picture of the domain's risk profile while keeping the solution accessible and reproducible. The enrichment scope was intentionally limited to provide sufficient context for confident decision-making without introducing excessive noise.

In [10]:
load_dotenv()

URLHAUS_KEY = os.getenv("URLHAUS_KEY")
VT_API_KEY  = os.getenv("VT_API_KEY")

# WHOIS enrichment
def enrich_whois(domain):
    try:
        result = whois.whois(domain)
        return {
            "whois_registrar": result.registrar,
            "whois_creation_date": str(result.creation_date),
            "whois_expiration_date": str(result.expiration_date),
            "whois_country": result.country,
            "whois_org": result.org,
        }
    except Exception as e:
        return {"whois_error": str(e)}

# URLhaus enrichment
def enrich_urlhaus(domain):
    try:
        response = requests.post(
            "https://urlhaus-api.abuse.ch/v1/host/",
            headers={"Auth-Key": URLHAUS_KEY},
            data={"host": domain},
            timeout=10
        )
        result = response.json()

        if result.get("query_status") == "ok":
            # Collecting all the unique malware family tags across all reported URLs
            all_tags = []
            for url in result.get("urls", []):
                if url.get("tags"):
                    all_tags.extend(url["tags"])

            return {
                "urlhaus_verdict": "malicious",
                "urlhaus_url_count": result.get("url_count", 0),
                "urlhaus_first_seen": result.get("firstseen"),
                "urlhaus_reference": result.get("urlhaus_reference"),
                "urlhaus_malware_families": ", ".join(list(set(all_tags))[:10]),
                "urlhaus_spamhaus_dbl": result.get("blacklists", {}).get("spamhaus_dbl"),
            }
        else:
            return {"urlhaus_verdict": "not found"}

    except Exception as e:
        return {"urlhaus_error": str(e)}

# VirusTotal enrichment
def enrich_virustotal(domain):
    try:
        response = requests.get(
            f"https://www.virustotal.com/api/v3/domains/{domain}",
            headers={"x-apikey": VT_API_KEY},
            timeout=10
        )
        result = response.json()
        attrs  = result["data"]["attributes"]
        stats  = attrs.get("last_analysis_stats", {})

        return {
            "vt_reputation": attrs.get("reputation"),
            "vt_malicious": stats.get("malicious", 0),
            "vt_suspicious": stats.get("suspicious", 0),
            "vt_harmless": stats.get("harmless", 0),
            "vt_undetected": stats.get("undetected", 0),
            "vt_verdict": (
                            "malicious" if stats.get("malicious", 0) > 0
                            else "suspicious" if stats.get("suspicious", 0) > 0
                            else "clean"
            ),
        }
    except Exception as e:
        return {"vt_error": str(e)}

# Running all three enrichment sources and combining them
def enrich_domain(domain):
    print(f"Enriching domain: {domain}")

    whois_data = enrich_whois(domain)
    urlhaus_data = enrich_urlhaus(domain)
    vt_data = enrich_virustotal(domain)

    print("\nSummary of enrichment:\n")
    urlhaus_data = enrich_urlhaus(domain)
    print(f" URLhaus verdict: {urlhaus_data.get('urlhaus_verdict')}")
    vt_data = enrich_virustotal(domain)
    print(f" VirusTotal verdict: {vt_data.get('vt_verdict')}")

    print("\nEnrichment Results:")
    return {
        "alert_id": alert_id,
        "event_creation_time" : event_time,
        "enriched_domain": domain,
        "dns_resolved_ips": dns_response_name,
        "enriched_at": datetime.now(UTC).strftime("%Y-%m-%d %H:%M:%S UTC"),

        "whois": whois_data,
        "urlhaus": urlhaus_data,
        "virustotal": vt_data,
    }

# fetching data from the alert table
alert_row = df_alert.first()

domain = alert_row["dns_query_name"]
alert_id = alert_row["alert_id"]
dns_response_name = alert_row["dns_response_name"]
event_time = alert_row["event_creation_time"]

enrichment_result = enrich_domain(domain)

print(json.dumps(enrichment_result, indent=4, default=str))

Enriching domain: www.mediafire.com

Summary of enrichment:

 URLhaus verdict: malicious
 VirusTotal verdict: clean

Enrichment Results:
{
    "alert_id": "725c19b21612dfdf94eef4bdb1ab157fac495183925056984a3c4aaafbbfc53c",
    "event_creation_time": "2023-01-27 11:30:14.523",
    "enriched_domain": "www.mediafire.com",
    "dns_resolved_ips": "::ffff:104.16.54.48;::ffff:104.16.53.48;",
    "enriched_at": "2026-02-25 02:42:44 UTC",
    "whois": {
        "whois_registrar": "Cloudflare, Inc.",
        "whois_creation_date": "2002-08-11 15:31:16+00:00",
        "whois_expiration_date": "2031-08-11 15:31:16+00:00",
        "whois_country": "US",
        "whois_org": "DATA REDACTED"
    },
    "urlhaus": {
        "urlhaus_verdict": "malicious",
        "urlhaus_url_count": "678",
        "urlhaus_first_seen": "2019-01-14 15:46:01 UTC",
        "urlhaus_reference": "https://urlhaus.abuse.ch/host/www.mediafire.com/",
        "urlhaus_malware_families": "Formbook, minecraft token stealer, dro

### From an analyst’s perspective, the final verdict would be:

- WHOIS indicates the domain was registered in 2002, registrar is Cloudflare Inc., located in the US. Registrant identity is hidden behind privacy protection (DATA REDACTED). While the domain age itself is not suspicious, the hidden ownership is worth noting.

- VirusTotal rates `www.mediafire.com` as clean (reputation score: 62) because it is a legitimate file sharing platform used by millions of people. VirusTotal scores the domain itself, not the individual files hosted on it.

- URLhaus tells a different story, 678 malicious URLs have been tracked on this domain since 2019, with active campaigns as recently as February 2026 distributing malware families including Formbook and rat.

- While `www.mediafire.com` itself is a legitimate platform, it has a well-documented history of being abused by threat actors for payload hosting. Its trusted reputation often allows malicious files to bypass simple domain-based blocklists, which aligns with the macro-based phishing scenario we’re investigating.

- What would the analyst do next?
    - Isolate the affected host `win-host-ctus-attack-range-212` and investigate whether a payload was successfully downloaded from mediafire.com around `2023-01-27 11:30:14 UTC`. That would confirm whether this was just a DNS lookup or an actual compromise attempt.

## Summary

- In the first part, I performed an analysis of 28,017 Sysmon events to detect macro-based phishing activity based on the hypothesis that Microsoft Office applications making unprompted DNS queries indicate a malicious macro beaconing to a C2 server.

- In the second part, I parsed the data into a structured Silver layer using `get_json_object()` to handle mixed event types cleanly. The detection query returned 8 results, 7 legitimate Microsoft telemetry domains and 1 suspicious domain (`www.mediafire.com`).

- In the third part, I normalized the findings to OSSEM, packaged them into a high-fidelity alert with MITRE ATT&CK mappings, and enriched them using WHOIS, URLhaus, and VirusTotal. URLhaus confirmed 678 malicious URLs on mediafire.com since 2019. The final assessment is high-confidence suspicious. A single DNS query isn’t enough to confirm a full compromise, but the behavior is unusual enough that it should be investigated right away.

### Resources / References Used:
- https://www.ultimatewindowssecurity.com/securitylog/encyclopedia/event.aspx?eventid=90022
- https://spark.apache.org/docs/latest/sql-getting-started.html#running-sql-queries-programmatically
- https://dlcdn.apache.org/spark/docs/3.4.3/api/python/reference/pyspark.sql/api/pyspark.sql.functions.sha2.html
- https://github.com/OTRF/OSSEM-DD/blob/afd9b27897346dfa3b3f43d2d403c3f5c5f86214/windows/sysmon/events/event-22.yml
- https://ossemproject.com/cdm/entities/dns.html
- https://www.elastic.co/docs/reference/ecs/ecs-dns
- https://pypi.org/project/python-whois/
- https://urlhaus-api.abuse.ch/
- https://docs.virustotal.com/reference/domain-info
- https://www.checkpoint.com/cyber-hub/threat-prevention/what-is-malware/asyncrat-malware-explained/

### Extra Part: Some Additional Investigation:

Tried to look if the winwird.exe made any network connections that we suspected to be made by "win-host-ctus-attack-range-212"

In [11]:
# Filtered specifically for winword.exe network connections
df_winword_connections = df_bronze.filter(
    (F.col("EventCode") == "3") &
    (F.col("Computer") == "win-host-ctus-attack-range-212") &
    (F.lower(F.get_json_object(F.col("_raw"), "$.Image")).contains("winword"))
).select(
    F.get_json_object(F.col("_raw"), "$.UtcTime").alias("event_time"),
    F.get_json_object(F.col("_raw"), "$.Image").alias("process_image"),
    F.get_json_object(F.col("_raw"), "$.DestinationIp").alias("destination_ip"),
    F.get_json_object(F.col("_raw"), "$.DestinationPort").alias("destination_port"),
    F.get_json_object(F.col("_raw"), "$.DestinationHostname").alias("destination_hostname"),
)

print(f"winword.exe network connections: {df_winword_connections.count()}")
df_winword_connections.toPandas()

winword.exe network connections: 46


,event_time,process_image,destination_ip,destination_port,destination_hostname
0,2023-01-27 11:30:14.007,C:\Program Files\Microsoft Office\root\Office16\WINWORD.EXE,52.109.8.44,443,-
1,2023-01-27 11:30:13.965,C:\Program Files\Microsoft Office\root\Office16\WINWORD.EXE,184.31.205.109,443,a184-31-205-109.deploy.static.akamaitechnologies.com
2,2023-01-27 11:30:13.963,C:\Program Files\Microsoft Office\root\Office16\WINWORD.EXE,52.113.194.132,443,-
3,2023-01-27 11:30:13.603,C:\Program Files\Microsoft Office\root\Office16\WINWORD.EXE,52.109.2.151,443,-
4,2023-01-27 11:30:14.298,C:\Program Files\Microsoft Office\root\Office16\WINWORD.EXE,72.21.91.29,80,-
5,2023-01-27 11:30:14.226,C:\Program Files\Microsoft Office\root\Office16\WINWORD.EXE,52.109.8.84,443,-
6,2023-01-27 11:30:14.787,C:\Program Files\Microsoft Office\root\Office16\WINWORD.EXE,172.64.155.188,80,-
7,2023-01-27 11:30:14.750,C:\Program Files\Microsoft Office\root\Office16\WINWORD.EXE,104.16.54.48,443,-
8,2023-01-27 11:30:14.528,C:\Program Files\Microsoft Office\root\Office16\WINWORD.EXE,104.16.54.48,443,-
9,2023-01-27 11:30:23.968,C:\Program Files\Microsoft Office\root\Office16\WINWORD.EXE,104.46.162.226,443,-


Three such instances were identified and subsequently filtered out.

In [12]:
# This confirms winword.exe connected to the mediafire.com IP
# We had dns_response_name showed ::ffff:104.16.54.48 which maps to 104.16.54.48
df_mediafire_connection = df_winword_connections.filter(
    F.col("destination_ip") == "104.16.54.48"
)

print(f"Confirmed connections to mediafire.com IP: {df_mediafire_connection.count()}")
df_mediafire_connection.toPandas()

Confirmed connections to mediafire.com IP: 3


,event_time,process_image,destination_ip,destination_port,destination_hostname
0,2023-01-27 11:30:14.750,C:\Program Files\Microsoft Office\root\Office16\WINWORD.EXE,104.16.54.48,443,-
1,2023-01-27 11:30:14.528,C:\Program Files\Microsoft Office\root\Office16\WINWORD.EXE,104.16.54.48,443,-
2,2023-01-27 11:30:50.074,C:\Program Files\Microsoft Office\root\Office16\WINWORD.EXE,104.16.54.48,443,-


In [13]:
# Reviewed EventCode 11 (File Create) activity for winword.exe on the same host to identify potential payload drops, as malware typically writes a file to disk before execution and cleanup.

df_file_create = df_bronze.filter(
    (F.col("EventCode") == "11") &
    (F.col("Computer") == "win-host-ctus-attack-range-212") &
    (F.lower(F.get_json_object(F.col("_raw"), "$.Image")).contains("winword"))
).select(
    F.get_json_object(F.col("_raw"), "$.UtcTime").alias("event_time"),
    F.get_json_object(F.col("_raw"), "$.Image").alias("process_image"),
    F.get_json_object(F.col("_raw"), "$.TargetFilename").alias("file_created"),
)

print(f"Files created by winword.exe: {df_file_create.count()}")
df_file_create.toPandas()

Files created by winword.exe: 2


,event_time,process_image,file_created
0,2023-01-27 11:30:49.915,C:\Program Files\Microsoft Office\Root\Office16\WINWORD.EXE,C:\Users\Administrator\AppData\Roaming\Microsoft\Office\Recent\asyncrat.doc.LNK
1,2023-01-27 11:30:49.853,C:\Program Files\Microsoft Office\Root\Office16\WINWORD.EXE,C:\Users\Administrator\AppData\Roaming\Microsoft\Office\Recent\asyncrat.doc.LNK


Interesting, winword.exe created a shortcut file named `asyncrat.doc.LNK`, confirming the malicious document was named asyncrat.doc and is associated with the AsyncRAT Remote Access Trojan family.

In [14]:
# Reviewed EventCode 23 (File Delete) for winword.exe on the same host
df_file_delete = df_bronze.filter(
    (F.col("EventCode") == "23") &
    (F.col("Computer") == "win-host-ctus-attack-range-212") &
    (F.lower(F.get_json_object(F.col("_raw"), "$.Image")).contains("winword"))
).select(
    F.get_json_object(F.col("_raw"), "$.UtcTime").alias("event_time"),
    F.get_json_object(F.col("_raw"), "$.Image").alias("process_image"),
    F.get_json_object(F.col("_raw"), "$.TargetFilename").alias("file_deleted"),
)

print(f"Files deleted by winword.exe: {df_file_delete.count()}")
df_file_delete.toPandas()

Files deleted by winword.exe: 113


,event_time,process_image,file_deleted
0,2023-01-27 11:30:15.034,C:\Program Files\Microsoft Office\Root\Office16\WINWORD.EXE,C:\Users\Administrator\AppData\Local\Microsoft\Windows\INetCache\Content.MSO\1ED02ABF.wmf
1,2023-01-27 11:30:16.031,C:\Program Files\Microsoft Office\Root\Office16\WINWORD.EXE,C:\Users\Administrator\AppData\Local\Microsoft\Windows\INetCache\Content.MSO\A9AA4225.wmf
2,2023-01-27 11:30:19.254,C:\Program Files\Microsoft Office\Root\Office16\WINWORD.EXE,C:\Users\Administrator\AppData\Local\Microsoft\Office\OTele\winword.exe.db-journal
3,2023-01-27 11:30:19.239,C:\Program Files\Microsoft Office\Root\Office16\WINWORD.EXE,C:\Users\Administrator\AppData\Local\Microsoft\Office\OTele\winword.exe.db-journal
4,2023-01-27 11:30:49.994,C:\Program Files\Microsoft Office\Root\Office16\WINWORD.EXE,C:\Users\Administrator\AppData\Local\Microsoft\Windows\INetCache\Content.MSO\E99D19E2.png
...,...,...,...
108,2023-01-27 11:31:03.147,C:\Program Files\Microsoft Office\Root\Office16\WINWORD.EXE,C:\Users\ADMINI~1\AppData\Local\Temp\2\cab5910.tmp
109,2023-01-27 11:31:03.146,C:\Program Files\Microsoft Office\Root\Office16\WINWORD.EXE,C:\Users\ADMINI~1\AppData\Local\Temp\2\cab5928.tmp
110,2023-01-27 11:31:03.146,C:\Program Files\Microsoft Office\Root\Office16\WINWORD.EXE,C:\Users\ADMINI~1\AppData\Local\Temp\2\TCD5976.tmp\Content.inf
111,2023-01-27 11:31:03.144,C:\Program Files\Microsoft Office\Root\Office16\WINWORD.EXE,C:\Users\ADMINI~1\AppData\Local\Temp\2\TCD5974.tmp\Content.inf


Seeing `113` file deletions within less than a minute, between `11:30:15` and `11:31:03`, immediately stood out. That volume of deletions in such a short window isn’t typical and definitely raises concern, especially in the context of suspected malware activity.

#### Suspected Full Attack Chain :
1. The user opened `asyncrat.doc` on `win-host-ctus-attack-range-212` around `11:30:49`.
2. Shortly after, a DNS query was made to `www.mediafire.com`, suggesting the macro may have initiated outbound communication. 
3. winword.exe then established multiple outbound connections to `104.16.54.48` (MediaFire), including one at `11:30:50`, which strengthens the likelihood of follow-on activity.
4. Based on the timing and known abuse patterns, it is likely that an AsyncRAT payload was downloaded and potentially executed, though this would require some additional investigation.
5. Within the window of `11:30:15` to `11:31:03`, 113 files were deleted. I understand that the file deletion alone does not confirm malware, but the timing overlaps with the suspected execution window, and is consistent with cleanup behavior seen in some malware families.